In [2]:
from dotenv import load_dotenv
import os 

load_dotenv()

os.environ['LANGSMITH_PROJECT']  ="skn15-1"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")


True

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="./data/아리계곡_통합.csv")
docs = loader.load()


In [10]:
len(docs)

2697

In [12]:
docs[0].page_content

'지점명: 강남점\ndate: 2025년 7월 22일 화요일\nnickname: 임구임구\nreview_content: 닭을 샤브샤브처럼 먹는것도 새롭고 메뉴가 다 맛있어서 이것저것 엄청 먹어봤어요!소곱창순두부전걸도 맛있었는데 흡입하니라 사진이 없는정도 ㅋㅋㅋ강남 술 모임 하실때 아주 괜찮습니다👍🏻👍🏻더보기\nwait_time: 바로 입장'

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [14]:
embedding = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embedding = embedding)

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [26]:
prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model="gpt-4.1", temperature=0)

retriever = db.as_retriever()

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | prompt | model | StrOutputParser()

In [27]:
output = chain.invoke("분위기가 어두운 편인가?")

In [28]:
import pprint

pprint.pprint(output)

('문맥에 따르면, 리뷰에서 "분위기 넘 좋고", "분위기도 좋고 데이트하기 너무 좋아요!"와 같은 긍정적인 표현이 사용되고 있습니다. '
 '하지만 "분위기가 어둡다"는 직접적인 언급이나 어두운 분위기를 암시하는 내용은 없습니다. 오히려 친구들과 좋은 시간을 보내거나 데이트하기 '
 '좋다는 점에서 밝고 쾌적한 분위기임을 시사합니다.\n'
 '\n'
 '따라서, 문맥만을 고려할 때 **분위기가 어두운 편이라고 볼 근거는 없습니다**.')


In [30]:
from pydantic import BaseModel, Field


class QueryGenerationOutput(BaseModel):
    queries: list[str] = Field(..., description="검색 쿼리 목록")


query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
    query_generation_prompt
    | model.with_structured_output(QueryGenerationOutput)
    | (lambda x: x.queries)
)

In [31]:
multi_query_rag_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

In [33]:
output = multi_query_rag_chain.invoke("주위사람에게 추천할 여부?")

In [34]:
pprint.pprint(output)

('문맥에 따르면, 여러 리뷰에서 "친구들이랑 오기 좋을 것 같아요", "지인추천으로 한번 왔다가", "건대오시면 강력추천드립니다" 등 '
 '주위사람에게 추천하거나 추천받았다는 내용이 반복적으로 언급되고 있습니다. 따라서, 주위사람에게 추천할 만하다고 볼 수 있습니다.')


In [35]:
os.environ['LANGCHAIN_API_KEY']

'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'

In [36]:
from langchain_core.documents import Document


def reciprocal_rank_fusion( retriever_outputs: list[list[Document]],  k: int = 60 ) -> list[str]:
    # 각 문서의 콘텐츠(문자열)와 그 점수의 매핑을 저장하는 딕셔너리 준비
    content_score_mapping = {}

    # 검색 쿼리마다 반복
    for docs in retriever_outputs:
        # 검색 결과의 문서마다 반복
        for rank, doc in enumerate(docs):
            content = doc.page_content

            # 처음 등장한 콘텐츠인 경우 점수를 0으로 초기화
            if content not in content_score_mapping:
                content_score_mapping[content] = 0

            # (1 / (순위 + k)) 점수를 추가
            content_score_mapping[content] += 1 / (rank + k)

    # 점수가 큰 순서로 정렬
    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # noqa
    return [content for content, _ in ranked]

In [37]:
rag_fusion_chain = {
    "question": RunnablePassthrough(),
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,
} | prompt | model | StrOutputParser()


In [39]:
rag_fusion_chain.invoke("가게에서 개선해야 하는 것은?")

'주어진 문맥에서는 대부분의 리뷰가 매장 분위기, 인테리어, 메뉴, 직원 친절도 등 긍정적인 평가를 하고 있습니다. 또한 "바로 입장" 또는 "10분 이내" 등 대기 시간도 짧은 편입니다. 다만, 종각점에서 "30분 이내" 대기 시간이 언급된 점을 볼 때, 일부 시간대나 지점(특히 종각점)에서는 대기 시간이 다소 길어질 수 있습니다.\n\n따라서, 문맥만을 고려할 때 가게에서 개선해야 할 점은 **일부 지점(특히 종각점)의 대기 시간 관리**입니다.'

In [40]:
from langchain_community.retrievers import TavilySearchAPIRetriever


In [43]:
langchain_document_retriever = retriever.with_config({'run_name' : 'langchain_document_retriver'})
web_retriever = TavilySearchAPIRetriever(k=10).with_config({'run_name' : 'web_retriver'})

In [47]:
from enum import Enum
from pydantic import BaseModel, Field

class Route(str, Enum):
    langchain_document = 'langchain_document'
    web = "web"

class RouteOutput(BaseModel):
    route: Route

route_prompt  =  ChatPromptTemplate.from_template("""   
질문에 답변하기 위한 적절한 Retriever를 선택하세요.

질문: {question}
""")

route_chain = (
    route_prompt | model.with_structured_output(RouteOutput) 
    | (lambda x : x.route)
)

In [55]:
def routed_retriever(inp):
    question = inp['question']
    route = inp['route']

    if route == Route.langchain_document:
        return langchain_document_retriever.invoke(question)
    elif route == Route.web:
        return web_retriever.invoke(question)

    raise ValueError(f"Unkown route: {route}")


In [56]:
route_rag_chain = ({
    'question' : RunnablePassthrough(),
    "route" : route_chain
}
| RunnablePassthrough.assign(context=routed_retriever)
| prompt | model | StrOutputParser())

In [58]:
route_rag_chain.invoke("아리계속에서 전통주와 막걸리 차이가 뭐야?")

'문맥에 따르면, **막걸리는 대표적인 전통주(탁주)** 중 하나입니다.  \n- **탁주**는 빚깔이 탁하고 흐린 술을 말하며, 막걸리가 대표적입니다.\n- **전통주**는 지역과 원료에 따라 다양한 술을 포함하는 넓은 개념이고, 막걸리는 그중 하나입니다.\n\n즉, **막걸리는 전통주(특히 탁주)의 한 종류**이고, 전통주는 막걸리를 포함한 다양한 한국의 전통 술 전체를 의미합니다.  \n따라서, **전통주와 막걸리의 차이**는  \n- 전통주는 막걸리를 포함한 모든 한국 전통 술을 의미하고,  \n- 막걸리는 그중에서 쌀로 빚고 탁한 빛깔을 가진 대표적인 술입니다.'